# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [13]:
def answer_one():
    import pandas as pd

    pd.set_option('display.max_rows',200)

    inputFile = 'C:/Users/dfitzgerald/Documents/MyCourses/IntrotoDataScienceinPython/coursedownloads/energy Indicators.xls'

    # There are 18 rows preceding the data set of interest, and 18 38 footer rows below the range of interest
    # the data of itntestes is in Columns C through F
    # apply meaningful column names
    df = pd.read_excel(inputFile,
                      skipfooter=38,
                      skiprows=18,
                      usecols='C:F',
                      names =['Country', 'Energy Supply','Energy Supply per Capita','% Renewable'])

    # get rid of numbers (footnotes) in the country names
    # set the index of the dataframe to Country
    # perform specific requested cleaning 
    energy = (df.replace({'Country':'[0-9]+'},{'Country':''}, regex=True)
                .replace({'Country': ' \\([a-zA-Z0-9_ ]+\\)'},{'Country': ''}, regex=True)
                .set_index(['Country'])
                .rename(index={'Republic of Korea': 'South Korea',
                              'United States of America': 'United States',
                              'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
                              'China, Hong Kong Special Administrative Region': 'Hong Kong'}))
    # world_bank.csv
    inputFile = 'C:/Users/dfitzgerald/Documents/MyCourses/IntrotoDataScienceinPython/coursedownloads/world_bank.csv'
    df = pd.read_csv(inputFile,
                     skiprows=4)

    # only keep the columns of interest
    columns_to_keep = ['Country Name',
                      'Country Code',
                      'Indicator Name',
                      'Indicator Code',
                      '2006',
                      '2007',
                      '2008',
                      '2009',
                      '2010',
                      '2011',
                      '2012',
                      '2013',
                      '2014',
                      '2015']

    df = df[columns_to_keep]

    # rename the Country Name to Country to allow easier alignment on merge
    for col in df.columns:
        if col=='Country Name':
            df.rename(columns={col:'Country'}, inplace=True)

            # set the index to the Country column
    df = df.set_index('Country')

    # clarify and update coutnry names
    GDP = df.rename(index={"Korea, Rep.": "South Korea", 
                          "Iran, Islamic Rep.": "Iran",
                         "Hong Kong SAR, China": "Hong Kong"})
    inputFile = 'C:/Users/dfitzgerald/Documents/MyCourses/IntrotoDataScienceinPython/coursedownloads/scimagojr-3.xlsx'

    # There are 18 rows preceding the data set of interest, and 18 38 footer rows below the range of interest
    # the data of itntestes is in Columns C through F
    # apply meaningful column names
    df =  pd.read_excel(inputFile,
                           index_col=1)
    ScimEn = df[df['Rank']< 16]
    
    # join the energy and GDP dataframes together on Country names
    energy_GDP = pd.merge(energy, GDP, 
                     how='inner', 
                     left_index=True,
                     right_index=True)

    energy_GDP_ScimEn = pd.merge(energy_GDP, ScimEn, 
                       how='inner', 
                       left_index=True, 
                       right_index=True)

    energy_GDP_ScimEn.head()

    # energy_GDP_ScimEn = energy_GDP_ScimEn.set_index('Country')
    # newerdf.head()
    # newerdf['Country'] = newerdf.index

    # final list of columns for the assignment questions dataframe
    columns_to_keep = ['Rank', 
                       'Documents', 
                       'Citable documents', 
                       'Citations', 
                       'Self-citations', 
                       'Citations per document', 
                       'H index', 
                       'Energy Supply', 
                       'Energy Supply per Capita', 
                       '% Renewable', 
                       '2006', 
                       '2007', 
                       '2008', 
                       '2009', 
                       '2010', 
                       '2011', 
                       '2012', 
                       '2013', 
                       '2014', 
                       '2015']
    returndf = energy_GDP_ScimEn[columns_to_keep]
    
    return returndf

returndf = answer_one()
returndf.head(15)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5386,231,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,12149,59,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6,17899,17620,215003,40930,12.01,149,10431,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1,127050,126767,597237,411683,4.70,138,127191,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
France,9,13153,12973,130632,28601,9.93,114,10597,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
Germany,7,17027,16831,140566,27426,8.26,126,13261,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,33195,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Iran,13,8896,8819,57470,19125,6.46,72,9172,119,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Italy,11,10964,10794,111850,26661,10.20,106,6530,109,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12


In [4]:
import pandas as pd

# world_bank.csv
inputFile = 'C:/Users/dfitzgerald/Documents/MyCourses/IntrotoDataScienceinPython/coursedownloads/world_bank.csv'
df = pd.read_csv(inputFile,
                 skiprows=4)

# only keep the columns of interest
columns_to_keep = ['Country Name',
                  'Country Code',
                  'Indicator Name',
                  'Indicator Code',
                  '2006',
                  '2007',
                  '2008',
                  '2009',
                  '2010',
                  '2011',
                  '2012',
                  '2013',
                  '2014',
                  '2015']

df = df[columns_to_keep]

# rename the Country Name to Country to allow easier alignment on merge
for col in df.columns:
    if col=='Country Name':
        df.rename(columns={col:'Country'}, inplace=True)

        # set the index to the Country column
df = df.set_index('Country')

# clarify and update coutnry names
GDP = df.rename(index={"Korea, Rep.": "South Korea", 
                      "Iran, Islamic Rep.": "Iran",
                     "Hong Kong SAR, China": "Hong Kong"})


In [5]:
inputFile = 'C:/Users/dfitzgerald/Documents/MyCourses/IntrotoDataScienceinPython/coursedownloads/scimagojr-3.xlsx'

# There are 18 rows preceding the data set of interest, and 18 38 footer rows below the range of interest
# the data of itntestes is in Columns C through F
# apply meaningful column names
df =  pd.read_excel(inputFile,
                       index_col=1)
ScimEn = df[df['Rank']< 16]

In [8]:
# join the energy and GDP dataframes together on Country names
energy_GDP = pd.merge(energy, GDP, 
                 how='inner', 
                 left_index=True,
                 right_index=True)

energy_GDP_ScimEn = pd.merge(energy_GDP, ScimEn, 
                   how='inner', 
                   left_index=True, 
                   right_index=True)

energy_GDP_ScimEn.head()

# energy_GDP_ScimEn = energy_GDP_ScimEn.set_index('Country')
# newerdf.head()
# newerdf['Country'] = newerdf.index

# final list of columns for the assignment questions dataframe
columns_to_keep = ['Rank', 
                   'Documents', 
                   'Citable documents', 
                   'Citations', 
                   'Self-citations', 
                   'Citations per document', 
                   'H index', 
                   'Energy Supply', 
                   'Energy Supply per Capita', 
                   '% Renewable', 
                   '2006', 
                   '2007', 
                   '2008', 
                   '2009', 
                   '2010', 
                   '2011', 
                   '2012', 
                   '2013', 
                   '2014', 
                   '2015']
returndf = energy_GDP_ScimEn[columns_to_keep]
returndf

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5386,231,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,12149,59,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6,17899,17620,215003,40930,12.01,149,10431,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1,127050,126767,597237,411683,4.70,138,127191,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
France,9,13153,12973,130632,28601,9.93,114,10597,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
Germany,7,17027,16831,140566,27426,8.26,126,13261,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,33195,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Iran,13,8896,8819,57470,19125,6.46,72,9172,119,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Italy,11,10964,10794,111850,26661,10.20,106,6530,109,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12


In [7]:

# energy_GDP_ScimEn = energy_GDP_ScimEn.set_index('Country')
# newerdf.head()
# newerdf['Country'] = newerdf.index

# final list of columns for the assignment questions dataframe
columns_to_keep = ['Rank', 
                   'Documents', 
                   'Citable documents', 
                   'Citations', 
                   'Self-citations', 
                   'Citations per document', 
                   'H index', 
                   'Energy Supply', 
                   'Energy Supply per Capita', 
                   '% Renewable', 
                   '2006', 
                   '2007', 
                   '2008', 
                   '2009', 
                   '2010', 
                   '2011', 
                   '2012', 
                   '2013', 
                   '2014', 
                   '2015']
returndf = energy_GDP_ScimEn[columns_to_keep]
returndf

NameError: name 'energy_GDP_ScimEn' is not defined

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [15]:
# join the energy and GDP dataframes together on Country names
energy_GDP_len = pd.merge(energy, GDP, 
                 how='outer', 
                 left_index=True,
                 right_index=True)

energy_GDP_ScimEn_len = pd.merge(energy_GDP_len, ScimEn, 
                   how='outer', 
                   left_index=True, 
                   right_index=True)

energy_GDP_ScimEn_len.head()
len(energy_GDP_ScimEn_len)-len(energy_GDP_ScimEn)

290

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)
​
*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [72]:
energy_GDP_ScimEn.loc['Australia']

Energy Supply                                                   5386
Energy Supply per Capita                                         231
% Renewable                                                  11.8108
Country Code                                                     AUS
Indicator Name              GDP at market prices (constant 2010 US$)
Indicator Code                                        NY.GDP.MKTP.KD
2006                                                     1.02194e+12
2007                                                     1.06034e+12
2008                                                     1.09964e+12
2009                                                     1.11965e+12
2010                                                     1.14225e+12
2011                                                     1.16943e+12
2012                                                     1.21191e+12
2013                                                     1.24148e+12
2014                              

In [10]:
import numpy as np
def avg_gdp(row):
    data = row[['2006', 
                '2007', 
                '2008', 
                '2009', 
                '2010', 
                '2011', 
                '2012', 
                '2013', 
                '2014', 
                '2015']]
    return pd.Series({'avg gdp': np.average(data)})

In [25]:
test_df = answer_one()
columns_to_keep = ['2006', 
                   '2007', 
                   '2008', 
                   '2009', 
                   '2010', 
                   '2011', 
                   '2012', 
                   '2013', 
                   '2014', 
                   '2015']
returndf = test_df[columns_to_keep]
returndf.mean(axis=1, skipna=True)
#test_df.mean(axis=1, skipna= True)
#energy_GDP_ScimEn.mean(axis=1, skipna=True)

# energy_GDP_ScimEn.apply(avg_gdp, axis=1)

Country
Australia             1.164043e+12
Brazil                2.189794e+12
Canada                1.660647e+12
China                 6.348609e+12
France                2.681725e+12
Germany               3.493025e+12
India                 1.769297e+12
Iran                  4.441558e+11
Italy                 2.120175e+12
Japan                 5.542208e+12
South Korea           1.106715e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
United Kingdom        2.487907e+12
United States         1.536434e+13
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [47]:
df = answer_one()
#df.head()
df4 = df[df['Rank']==6]
x = df4['2015']-df4['2006']
df4.head()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Canada,6,17899,17620,215003,40930,12.01,149,10431,296,61.94543,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12


### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*